In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
G = nx.Graph()

nodes = list(range(1, 16))
G.add_nodes_from(nodes)

edges = [
    (9, 10), (9, 11), (9, 12), (9, 13), (9, 14),
   
    (9, 8), (8, 7), (7, 6),
  
    (15, 6),
  
    (6, 1), (6, 2), (6, 3), (6, 4), (6, 5),
   
    (1, 2), (1, 3), (1, 4), (1, 5),
    (2, 3), (2, 4), (2, 5),
    (3, 4), (3, 5),
    (4, 5)
]

G.add_edges_from(edges)

In [3]:

beta = 0.15
n = len(G.nodes())

W = nx.adjacency_matrix(G).todense()


eigenvalues = np.linalg.eigvals(W)
lambda_W = max(eigenvalues.real)

mu = np.ones((n, 1))

I = np.eye(n)
z_katz = np.linalg.inv(I - (1-beta)/lambda_W * W.T) @ (beta * mu)

z_katz = z_katz / np.sum(z_katz)

katz_dict = {node: float(z_katz[i, 0]) for i, node in enumerate(sorted(G.nodes()))}

print(f"\n{'Node':>6} {'Katz Centrality':>20}")
print("-" * 28)
for node in sorted(katz_dict.keys()):
    print(f"n{node:>5} {katz_dict[node]:>20.8f}")


  Node      Katz Centrality
----------------------------
n    1           0.11813975
n    2           0.11813975
n    3           0.11813975
n    4           0.11813975
n    5           0.11813975
n    6           0.13008034
n    7           0.04424717
n    8           0.03171920
n    9           0.04281112
n   10           0.02430259
n   11           0.02430259
n   12           0.02430259
n   13           0.02430259
n   14           0.02430259
n   15           0.03893046


In [4]:

beta = 0.15
n = len(G.nodes())
nodes = sorted(G.nodes())
max_iter = 100
tol = 1e-6

W = nx.adjacency_matrix(G, nodelist=nodes).todense()
degrees = np.array([G.degree(node) for node in nodes])
D_inv = np.diag(1.0 / degrees)

P = D_inv @ W

mu = np.ones(n) / n

z_old = np.ones(n) / n

iteration = 0
while iteration < max_iter:
    z_new = (1 - beta) * P.T @ z_old + beta * mu
    
    if np.linalg.norm(z_new - z_old) < tol:
        print(f"\nConverged in {iteration + 1} iterations")
        break
    
    z_old = z_new
    iteration += 1

z_pagerank = z_new / np.sum(z_new)

pagerank_dict = {node: float(z_pagerank[i]) for i, node in enumerate(nodes)}

print(f"\n{'Node':>6} {'PageRank Centrality':>22}")
print("-" * 30)
for node in sorted(pagerank_dict.keys()):
    print(f"n{node:>5} {pagerank_dict[node]:>22.8f}")


Converged in 68 iterations

  Node    PageRank Centrality
------------------------------
n    1             0.07467987
n    2             0.07467987
n    3             0.07467987
n    4             0.07467987
n    5             0.07467987
n    6             0.11445052
n    7             0.04861096
n    8             0.05814937
n    9             0.19404391
n   10             0.03748967
n   11             0.03748967
n   12             0.03748967
n   13             0.03748967
n   14             0.03748967
n   15             0.02389755


In [5]:

betas = [0, 0.25, 0.5, 0.75, 1]
beta_results = {}

for beta_val in betas:
    print(f"Computing for β = {beta_val:.2f}...", end=" ")
    
    n = len(G.nodes())
    nodes = sorted(G.nodes())
    max_iter = 100
    tol = 1e-6
    
    W = nx.adjacency_matrix(G, nodelist=nodes).todense()
    degrees = np.array([G.degree(node) for node in nodes])
    D_inv = np.diag(1.0 / degrees)
    P = D_inv @ W
    
    mu = np.ones(n) / n
    
    z_old = np.ones(n) / n
    
    iteration = 0
    while iteration < max_iter:
        z_new = (1 - beta_val) * P.T @ z_old + beta_val * mu
        
        if np.linalg.norm(z_new - z_old) < tol:
            print(f"Converged in {iteration + 1} iterations")
            break
        
        z_old = z_new
        iteration += 1
    
    z_new = z_new / np.sum(z_new)
    beta_results[beta_val] = {node: float(z_new[i]) for i, node in enumerate(nodes)}

Computing for β = 0.00... Computing for β = 0.25... Converged in 41 iterations
Computing for β = 0.50... Converged in 18 iterations
Computing for β = 0.75... Converged in 9 iterations
Computing for β = 1.00... Converged in 1 iterations


In [ ]:

node_a = 6
node_b = 9

betas_sorted = sorted(beta_results.keys())
differences = [beta_results[beta][node_a] - beta_results[beta][node_b] for beta in betas_sorted]


print(f"\n{'β':>8} {'PR(n'+str(node_a)+')':>15} {'PR(n'+str(node_b)+')':>15} {'Difference':>15}")
print("-" * 70)

for i, beta in enumerate(betas_sorted):
    pr_a = beta_results[beta][node_a]
    pr_b = beta_results[beta][node_b]
    diff = differences[i]
    print(f"{beta:>8.2f} {pr_a:>15.8f} {pr_b:>15.8f} {diff:>15.8f}")




       β          PR(n6)          PR(n9)      Difference
----------------------------------------------------------------------
    0.00      0.14594773      0.11619522      0.02975251
    0.25      0.11060445      0.19111627     -0.08051182
    0.50      0.10140400      0.16669233     -0.06528833
    0.75      0.08750662      0.12697790     -0.03947127
    1.00      0.06666667      0.06666667      0.00000000
